In [1]:
import numpy as np
import pandas as pd
import scipy.stats as st
import statsmodels.api as sm
from pandas.core.dtypes.common import is_numeric_dtype

data = pd.read_csv("data.csv")
data = data.set_index('projectID')
data = data.drop(columns=['drives'])
pd.set_option('display.float_format',  '{:,.3f}'.format)
data

,success,amount,female_creator,goal,words,backers_count,serial_entrepreneur,duration,category_art,category_comics,...,year_2020,year_2021,year_2022,media,sustainability,story,url,business,empathic,social
projectID,,,,,,,,,,,,,,,,,,,,,
3731940,0,203.000,0,"6,000.000",71,5,0,60.000,0,0,...,0,0,0,0,0,app go benefit trainer personnel look get fit ...,https://www.kickstarter.com/projects/fitnesspo...,1.408,0.000,8.451
3451605,1,"116,639.000",0,"10,000.000",497,2003,1,14.583,0,0,...,0,0,0,1,0,even miss kickstarter still get hand papillon ...,https://www.kickstarter.com/projects/kolossalg...,3.414,2.610,9.839
3722583,0,28.000,0,"6,000.000",443,6,0,30.000,0,1,...,0,0,0,1,0,journey blak blak blak officially entitle blak...,https://www.kickstarter.com/projects/myheart/b...,0.903,0.677,13.318
3745862,1,"1,846.000",1,"1,500.000",474,48,0,39.687,0,0,...,0,0,0,1,0,isabel jazz folk musical teach empathy metoo s...,https://www.kickstarter.com/projects/annawestb...,1.477,1.899,17.511
3735376,1,"1,001.000",1,"1,000.000",130,17,0,25.990,0,0,...,0,0,0,0,0,main stage kids head new york young performer ...,https://www.kickstarter.com/projects/mainstage...,0.000,0.769,16.154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4378823,1,"62,669.000",0,"28,000.000",837,232,1,30.000,0,0,...,0,0,1,1,0,book maui mauka makai companion book maui coas...,https://www.kickstarter.com/projects/danielsul...,0.239,1.912,6.810
4394019,1,"11,083.000",0,"10,000.000",167,80,0,26.377,0,0,...,0,0,1,1,0,myrcene ale co dream mike goergen cy higgins m...,https://www.kickstarter.com/projects/myrceneal...,1.198,3.593,9.581
4385597,1,"25,265.000",0,"25,000.000",138,73,0,14.147,0,0,...,0,0,1,1,0,firefly theatrical need help build theatre spa...,https://www.kickstarter.com/projects/fireflyth...,2.899,0.725,7.971


In [2]:
# Normalize data
# Box-Cox lambda values
# amount        0.0915075719918227
# goal          0.02059050704045948
# words         0.19404918878650212
# backers_count 0.010289890837174583
# duration      0.4346871019563134
# business      0.039097024902746386
# empathic      0.05775409842214908
# social        0.23771984812837177
# drives        0.22950948679659838

normData = data.copy()

for variable in ['amount', 'goal', 'words', 'backers_count', 'business', 'empathic', 'social']: # + drives
    minimum, secondMinimum = data[variable].drop_duplicates().nsmallest(2)
    if minimum == 0:
        normData[variable] += 0.5*secondMinimum

    normData[variable] = np.log10(normData[variable])
    normData = normData.rename(columns={variable: "norm_" + variable})

normData['duration'] = np.sqrt(normData['duration'])
normData = normData.rename(columns={'duration': 'norm_duration'})

normData.to_csv('normalisedData.csv')
normData

,success,norm_amount,female_creator,norm_goal,norm_words,norm_backers_count,serial_entrepreneur,norm_duration,category_art,category_comics,...,year_2020,year_2021,year_2022,media,sustainability,story,url,norm_business,norm_empathic,norm_social
projectID,,,,,,,,,,,,,,,,,,,,,
3731940,0,2.307,0,3.778,1.851,0.699,0,7.746,0,0,...,0,0,0,0,0,app go benefit trainer personnel look get fit ...,https://www.kickstarter.com/projects/fitnesspo...,0.165,-1.208,0.938
3451605,1,5.067,0,4.000,2.696,3.302,1,3.819,0,0,...,0,0,0,1,0,even miss kickstarter still get hand papillon ...,https://www.kickstarter.com/projects/kolossalg...,0.540,0.427,1.003
3722583,0,1.447,0,3.778,2.646,0.778,0,5.477,0,1,...,0,0,0,1,0,journey blak blak blak officially entitle blak...,https://www.kickstarter.com/projects/myheart/b...,-0.018,-0.131,1.132
3745862,1,3.266,1,3.176,2.676,1.681,0,6.300,0,0,...,0,0,0,1,0,isabel jazz folk musical teach empathy metoo s...,https://www.kickstarter.com/projects/annawestb...,0.185,0.292,1.249
3735376,1,3.000,1,3.000,2.114,1.230,0,5.098,0,0,...,0,0,0,0,0,main stage kids head new york young performer ...,https://www.kickstarter.com/projects/mainstage...,-1.256,-0.080,1.214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4378823,1,4.797,0,4.447,2.923,2.365,1,5.477,0,0,...,0,0,1,1,0,book maui mauka makai companion book maui coas...,https://www.kickstarter.com/projects/danielsul...,-0.531,0.295,0.847
4394019,1,4.045,0,4.000,2.223,1.903,0,5.136,0,0,...,0,0,1,1,0,myrcene ale co dream mike goergen cy higgins m...,https://www.kickstarter.com/projects/myrceneal...,0.098,0.563,0.992
4385597,1,4.403,0,4.398,2.140,1.863,0,3.761,0,0,...,0,0,1,1,0,firefly theatrical need help build theatre spa...,https://www.kickstarter.com/projects/fireflyth...,0.470,-0.104,0.914


In [3]:
# Table 1
mData = data[data['female_creator'] == 0]
fData = data[data['female_creator'] == 1]

categories = ['category_art', 'category_comics', 'category_crafts', 'category_dance', 'category_design',
             'category_fashion', 'category_film&video', 'category_food', 'category_games', 'category_journalism',
             'category_music', 'category_photography', 'category_publishing', 'category_technology', 'category_theater']

mMeans = []
fMeans = []
tValues = []
pValues = []
catNames = []
for category in categories:
    mCatData = mData[mData[category] == 1]
    fCatData = fData[fData[category] == 1]

    mMeans.append(np.mean(mCatData['success']))
    fMeans.append(np.mean(fCatData['success']))

    # Equal variance check
    mVar = np.var(mCatData['success'])
    fVar = np.var(fCatData['success'])

    bigger = np.maximum(mVar, fVar)
    smaller = np.minimum(mVar, fVar)

    equalVariance = False
    if bigger / smaller < 4:
        equalVariance = True

    # Perform two-sample t-test
    result = st.ttest_ind(a=mCatData['success'], b=fCatData['success'], equal_var=equalVariance)
    tValues.append(result[0])
    if result[1] >= 0.001:
        pValues.append(result[1])
    else:
        pValues.append("< 0.001")

    catNames.append(category.split('_')[1])

table1 = {"Category": catNames, "Male (mean)": mMeans, "Female (mean)": fMeans, "t-Value": tValues, "p-Value": pValues}
table1 = pd.DataFrame.from_dict(table1)
table1 = table1.set_index('Category')
table1.to_csv('Tables/success rate per category.csv')
table1

,Male (mean),Female (mean),t-Value,p-Value
Category,,,,
art,0.809,0.865,-2.389,0.017
comics,0.934,0.884,2.131,0.033
crafts,0.565,0.512,0.661,0.509
dance,0.750,0.846,-0.417,0.683
design,0.945,0.905,1.804,0.072
fashion,0.578,0.809,-5.433,< 0.001
film&video,0.542,0.623,-1.959,0.051
food,0.482,0.422,1.336,0.182
games,0.926,0.900,1.300,0.194


In [4]:
# Table 2 (part of table 7)
# mVar = np.var(mData['success'])
# fVar = np.var(fData['success'])
#
# bigger = np.maximum(mVar, fVar)
# smaller = np.minimum(mVar, fVar)
#
# equalVariance = False
# if bigger / smaller < 4:
#     equalVariance = True
#
# result = st.ttest_ind(a=mData['success'], b=fData['success'], equal_var=equalVariance)
# if result[1] < 0.001:
#     result[1] = "< 0.001"
#
# table2 = {'Variable': ['success'], 'Male (mean)': [np.mean(mData['success'])], 'Female (mean)': [np.mean(fData['success'])], 't-Value': [result[0]], 'p-Value': [result[1]]}
# table2 = pd.DataFrame.from_dict(table2)
# table2 = table2.set_index('Variable')
# table2.to_csv('Tables/gender breakdown.csv')
# table2

In [5]:
# Table 3
numVars = []
means = []
medians = []
minima = []
maxima = []
SDs = []

for variable in data.columns.tolist():
    varData = data[variable]
    if is_numeric_dtype(varData):
        numVars.append(variable)
        means.append(np.mean(varData))
        medians.append(np.median(varData))
        minima.append(np.min(varData))
        maxima.append(np.max(varData))
        SDs.append(np.std(varData))

table3 = {"Variable": numVars, "Mean": means, "Median": medians, "Minimum": minima, "Maximum": maxima, "SD": SDs}
table3 = pd.DataFrame.from_dict(table3)
table3 = table3.set_index("Variable")
table3.to_csv('Tables/descriptive statistics.csv')
table3

,Mean,Median,Minimum,Maximum,SD
Variable,,,,,
success,0.763,1.000,0.000,1.000,0.425
amount,"66,149.066","3,838.000",1.000,"272,707,148.204","2,984,093.960"
female_creator,0.327,0.000,0.000,1.000,0.469
goal,"19,854.147","4,000.000",1.000,"14,000,000.000","188,831.801"
words,408.228,305.000,2.000,"2,588.000",362.641
backers_count,345.802,65.000,1.000,"55,024.000","1,595.292"
serial_entrepreneur,0.252,0.000,0.000,1.000,0.434
duration,32.190,30.000,1.000,73.958,12.615
category_art,0.121,0.000,0.000,1.000,0.327


In [6]:
# Table 4
succData = normData[normData['success'] == 1]
failData = normData[normData['success'] == 0]

succMeans = []
failMeans = []
succMedians = []
failMedians = []

meanTValues = []
meanPValues = []

moodsVars = ['amount', 'goal', 'words', 'backers_count', 'duration', 'business', 'empathic', 'social'] # + drives
medianTValues = []
medianPValues = []


def cleanPValue(pValue):
    if pValue != "N/A":
        if pValue < 0.001:
            pValue = "< 0.001"
    return pValue


numVars_ = []
for var in numVars:
    if var in ['amount', 'goal', 'words', 'backers_count', 'business', 'empathic', 'social', 'duration']:
        var = "norm_" + var
    numVars_.append(var)
numVars = numVars_[:]

for variable in numVars[1:]:
    varDataSucc = succData[variable]
    varDataFail = failData[variable]

    # Compute means and medians
    succMeans.append(np.mean(varDataSucc))
    failMeans.append(np.mean(varDataFail))
    succMedians.append(np.median(varDataSucc))
    failMedians.append(np.median(varDataFail))

    # Check variance ratio
    succVar = np.var(varDataSucc)
    failVar = np.var(varDataFail)

    bigger = np.maximum(succVar, failVar)
    smaller = np.minimum(succVar, failVar)

    equalVariance = False
    if bigger / smaller < 4:
        equalVariance = True

    # Perform two sample t-test
    result = st.ttest_ind(a=varDataSucc, b=varDataFail, equal_var=equalVariance)
    meanTValues.append(result[0])
    meanPValues.append(cleanPValue(result[1]))

    # Perform Mood's median test
    if variable in moodsVars:
        result = st.median_test(varDataSucc, varDataFail)
    else:
        result = ["N/A", "N/A"]
    medianTValues.append(result[0])
    medianPValues.append(cleanPValue(result[1]))

table4 = {'Variable': numVars[1:], 'Success (mean)': succMeans, 'Failure (mean)': failMeans, 't-Value': meanTValues, 'p-Value': meanPValues}
table4 = pd.DataFrame.from_dict(table4)
table4 = table4.set_index('Variable')
table4.to_csv('Tables/mean per variable (success).csv')
table4

,Success (mean),Failure (mean),t-Value,p-Value
Variable,,,,
norm_amount,3.835,2.064,78.910,< 0.001
female_creator,0.328,0.324,0.330,0.741
norm_goal,3.433,3.998,-31.108,< 0.001
norm_words,2.504,2.274,22.815,< 0.001
norm_backers_count,2.071,0.783,82.970,< 0.001
serial_entrepreneur,0.324,0.019,46.108,< 0.001
norm_duration,5.359,6.234,-32.865,< 0.001
category_art,0.134,0.080,6.466,< 0.001
category_comics,0.136,0.035,12.578,< 0.001


In [7]:
# Table 4 (continued)
table4 = {'Variable': numVars[1:], 'Success (median)': succMedians, 'Failure (median)': failMedians, 'χ²-Value': medianTValues, 'p-Value': medianPValues}
table4 = pd.DataFrame.from_dict(table4)
table4 = table4.set_index('Variable')
table4.to_csv('Tables/median per variable (success).csv')
table4

,Success (median),Failure (median),χ²-Value,p-Value
Variable,,,,
norm_amount,3.787,2.233,N/A,N/A
female_creator,0.000,0.000,N/A,N/A
norm_goal,3.477,4.000,N/A,N/A
norm_words,2.538,2.291,N/A,N/A
norm_backers_count,2.017,0.699,N/A,N/A
serial_entrepreneur,0.000,0.000,N/A,N/A
norm_duration,5.477,5.571,N/A,N/A
category_art,0.000,0.000,N/A,N/A
category_comics,0.000,0.000,N/A,N/A


In [8]:
# Table 5
table5 = {
    'money': ['cash', 'fines', 'invest', 'irs', 'market', 'profit', 'rich', 'spend', 'usd', 'worth'],
    'emotion': ['admire', 'clinical depression', 'envies', 'furious', 'heartbreaking', 'misery', 'not a good idea', 'rage', 'tantrum', 'you\'d love'],
    'social': ['admittedly', 'consoling', 'ex-wife', 'ladies man', 'mob', 'papa', 'stalking', 'transgender', 'village', 'white guy'],
    # 'drives': ['abuser', 'chat', 'dear', 'denies', 'excelling', 'law', 'mamacita', 'perfection', 'sergeant', 'sultan']
}
table5 = pd.DataFrame.from_dict(table5)
table5.to_csv('Tables/category examples.csv', index=False)
table5

,money,emotion,social
0,cash,admire,admittedly
1,fines,clinical depression,consoling
2,invest,envies,ex-wife
3,irs,furious,ladies man
4,market,heartbreaking,mob
5,profit,misery,papa
6,rich,not a good idea,stalking
7,spend,rage,transgender
8,usd,tantrum,village
9,worth,you'd love,white guy


In [9]:
# Table 6
index = []
for variable in numVars:
    index.append(variable)
    index.append('')
table6 = pd.DataFrame(index=index, columns=numVars)

noVars = len(numVars)
for i in range(noVars):
    rValues = []
    pValues = []

    var1Data = normData[numVars[i]]
    for j in range(i + 1):
        if i == j:
            rValues.append(1.000)
            pValues.append('< 0.001')
        else:
            rValue, pValue = st.pearsonr(var1Data, normData[numVars[j]])
            rValues.append(rValue)
            if pValue < 0.001:
                pValue = '< 0.001'
            else:
                pValue = '{:.3f}'.format(pValue)
            pValues.append('(%s)' % pValue)

    for j in range(i + 1, noVars):
        rValues.append('')
        pValues.append('')

    table6.iloc[2*i] = rValues
    table6.iloc[2*i + 1] = pValues

table6.to_csv('Tables/correlation matrix.csv')
table6

,success,norm_amount,female_creator,norm_goal,norm_words,norm_backers_count,serial_entrepreneur,norm_duration,category_art,category_comics,...,month_december,year_2019,year_2020,year_2021,year_2022,media,sustainability,norm_business,norm_empathic,norm_social
success,1.000,,,,,,,,,,...,,,,,,,,,,
,< 0.001,,,,,,,,,,...,,,,,,,,,,
norm_amount,0.653,1.000,,,,,,,,,...,,,,,,,,,,
,(< 0.001),< 0.001,,,,,,,,,...,,,,,,,,,,
female_creator,0.004,-0.051,1.000,,,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,(0.025),(0.339),(< 0.001),(< 0.001),(< 0.001),(0.110),(< 0.001),(0.076),(< 0.001),(< 0.001),...,(0.605),(0.066),(0.388),(0.035),(0.052),(0.117),(< 0.001),< 0.001,,
norm_empathic,0.052,0.031,0.108,-0.031,0.194,0.015,-0.055,-0.008,0.036,0.003,...,0.001,-0.019,-0.011,0.024,0.003,0.052,0.018,0.047,1.000,
,(< 0.001),(0.004),(< 0.001),(0.005),(< 0.001),(0.179),(< 0.001),(0.464),(< 0.001),(0.773),...,(0.892),(0.090),(0.328),(0.028),(0.780),(< 0.001),(0.094),(< 0.001),< 0.001,
norm_social,-0.094,-0.103,0.100,0.081,0.114,-0.124,-0.131,0.072,-0.045,0.032,...,0.019,0.014,-0.014,0.009,-0.009,-0.118,0.007,0.136,0.279,1.000


In [10]:
# Table 7
mNormData = normData[normData['female_creator'] == 0]
fNormData = normData[normData['female_creator'] == 1]

mMeans = []
fMeans = []
tValues = []
pValues = []

numVars_ = numVars[:]
numVars_.remove('female_creator')
for variable in numVars_:
    mVarData = mNormData[variable]
    fVarData = fNormData[variable]

    # Calculate the means
    mMeans.append(np.mean(mVarData))
    fMeans.append(np.mean(fVarData))

    # Check for equal variance
    mVar = np.var(mVarData)
    fVar = np.var(fVarData)

    bigger = np.maximum(mVar, fVar)
    smaller = np.minimum(mVar, fVar)

    equalVariance = False
    if bigger / smaller < 4:
        equalVariance = True

    # Perform two-sample t-test
    result = st.ttest_ind(a=mVarData, b=fVarData, equal_var=equalVariance)
    tValues.append(result[0])
    if result[1] >= 0.001:
        pValues.append(result[1])
    else:
        pValues.append("< 0.001")

table7 = {"Variable": numVars_, "Male (mean)": mMeans, "Female (mean)": fMeans, "t-Value": tValues, "p-Value": pValues}
table7 = pd.DataFrame.from_dict(table7)
table7 = table7.set_index('Variable')
table7.to_csv('Tables/mean per variable (female_creator).csv')
table7

,Male (mean),Female (mean),t-Value,p-Value
Variable,,,,
success,0.762,0.766,-0.330,0.741
norm_amount,3.457,3.333,4.637,< 0.001
norm_goal,3.614,3.469,8.387,< 0.001
norm_words,2.464,2.421,4.580,< 0.001
norm_backers_count,1.809,1.679,6.889,< 0.001
serial_entrepreneur,0.278,0.199,7.851,< 0.001
norm_duration,5.570,5.556,0.541,0.588
category_art,0.067,0.234,-22.725,< 0.001
category_comics,0.140,0.054,11.902,< 0.001


In [11]:
# Determine which numerical variables are linearly related to amount
amountData = normData['norm_amount']

rValues = []
pValues = []
linears = []
for variable in numVars:
    rValue, pValue = st.pearsonr(amountData, normData[variable])
    rValues.append(rValue)
    pValues.append(pValue)
    linears.append(pValue < 0.05 and abs(rValue) >= 0.4)

correlations = pd.DataFrame()
correlations['Variable'] = numVars
correlations['r-Value'] = rValues
correlations['p-Value'] = pValues
correlations['Linear'] = linears
correlations = correlations.set_index('Variable')
correlations

,r-Value,p-Value,Linear
Variable,,,
success,0.653,0.000,True
norm_amount,1.000,0.000,True
female_creator,-0.051,0.000,False
norm_goal,0.187,0.000,False
norm_words,0.406,0.000,True
norm_backers_count,0.911,0.000,True
serial_entrepreneur,0.263,0.000,False
norm_duration,-0.188,0.000,False
category_art,-0.055,0.000,False


In [12]:
# Establish a baseline model
independentVariables = ['norm_words', 'norm_backers_count', 'media']

X = sm.add_constant(normData[independentVariables])
y = normData['norm_amount']
model = sm.OLS(y, X).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            norm_amount   R-squared:                       0.834
Model:                            OLS   Adj. R-squared:                  0.834
Method:                 Least Squares   F-statistic:                 1.401e+04
Date:                Tue, 01 Nov 2022   Prob (F-statistic):               0.00
Time:                        15:23:00   Log-Likelihood:                -5572.1
No. Observations:                8388   AIC:                         1.115e+04
Df Residuals:                    8384   BIC:                         1.118e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.8404      0.032     26.488      0.000       0.778       0.903
norm_words             0.0924      0.014      6.480      0.000       0.064       0.120
norm_backers_count     1.2326      0.007    167.032      0.000       1.218       1.247
media                  0.1970      0.018     11.081      0.000       0.162       0.232
==============================================================================
Omnibus:                      261.874   Durbin-Watson:                   1.956
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              708.533
Skew:                          -0.025   Prob(JB):                    1.39e-154
Kurtosis:                       4.423   Cond. No.                         22.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [13]:
independentVariables += ['norm_business']

X = sm.add_constant(normData[independentVariables])
y = normData['norm_amount']
model = sm.OLS(y, X).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            norm_amount   R-squared:                       0.834
Model:                            OLS   Adj. R-squared:                  0.834
Method:                 Least Squares   F-statistic:                 1.051e+04
Date:                Tue, 01 Nov 2022   Prob (F-statistic):               0.00
Time:                        15:23:00   Log-Likelihood:                -5571.9
No. Observations:                8388   AIC:                         1.115e+04
Df Residuals:                    8383   BIC:                         1.119e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.8359      0.032     25.855      0.000       0.772       0.899
norm_words             0.0951      0.015      6.464      0.000       0.066       0.124
norm_backers_count     1.2321      0.007    166.116      0.000       1.218       1.247
media                  0.1966      0.018     11.056      0.000       0.162       0.231
norm_business         -0.0069      0.009     -0.740      0.460      -0.025       0.011
==============================================================================
Omnibus:                      261.912   Durbin-Watson:                   1.956
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              708.633
Skew:                          -0.025   Prob(JB):                    1.33e-154
Kurtosis:                       4.423   Cond. No.                         23.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""